In [1]:
import pickle
import pandas as pd

with open('./results/HAS_e.pickle', 'rb') as f:
    HAS_e = pickle.load(f)
HAS_t = pd.read_parquet('./results/HAS_t.parquet')

In [2]:
from bertopic import BERTopic
from gensim.models.ldamulticore import LdaMulticore
from utils import e_variant

def load_ldas():
    r = {} # result
    for v in e_variant():
        r[v] = LdaMulticore.load(f'./results/models_lda/{v}')
    return r

def load_bertopics():
    path_prefix = './results/models_bertopic/'
    r = {}
    for v in e_variant():
        r[v] = {}
        r[v] = BERTopic.load(f'{path_prefix}{v}')
    return r

d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython'

In [3]:
ldas = load_ldas()
bertopics = load_bertopics()

In [4]:
from time import time
from gensim.corpora.dictionary import Dictionary
from utils import get_topics_lda, get_topics_bertopic, get_coherence, get_diversity

def evaluate_models(models, lda=False):
    r = []
    r_time = []
    for v in HAS_e:
        t_start = time()
        if lda:
            topics = get_topics_lda(models[v], HAS_e[v]['id2word'])
            texts = HAS_e[v]['T']
            dictionary = models[v].id2word
        else:
            topics = get_topics_bertopic(bertopics[v])
            analyzer = models[v].vectorizer_model.build_analyzer()
            texts = [analyzer(doc) for doc in HAS_e[v]['text']]
            dictionary = Dictionary(texts)
        c = get_coherence(
            topics=topics,
            texts=texts,
            dictionary=dictionary,
        )
        d = get_diversity(topics)
        t_end = time()
        r_time.append(t_end - t_start)
        r.append({
            'variant': v,
            'coherence': c,
            'diversity': d,
            'score': c * d
        })
    algorithm = 'lda' if lda else 'bertopic'
    HAS_t[f'{algorithm}_evaluation'] = pd.Series(r_time)
    print(r)
    return pd.DataFrame(r)

In [5]:
import pandas as pd

evaluation_lda = pd.DataFrame(evaluate_models(ldas, lda=True))
evaluation_bertopic = pd.DataFrame(evaluate_models(bertopics))

[{'variant': 'HWN', 'coherence': 0.41576517142692637, 'diversity': 0.5532467532467532, 'score': 0.23002073120502678}]
[{'variant': 'HWN', 'coherence': 0.40450850608995553, 'diversity': 1.0, 'score': 0.40450850608995553}]


In [6]:
HAS_t

,variant,segmentation,preprocess,lda_embedding,lda_training,bertopic_training,lda_evaluation,bertopic_evaluation
0,HWN,0.001003,2.34099,0.003,19.633259,7.961455,2.713751,1.761843


In [7]:
evaluation_lda

,variant,coherence,diversity,score
0,HWN,0.415765,0.553247,0.230021


In [8]:
evaluation_bertopic

,variant,coherence,diversity,score
0,HWN,0.404509,1.0,0.404509
